# Task 2: Transfer Learning with Pretrained Models for Pet Classification

# 1. Introduction and Setup

In this notebook, we'll use transfer learning with pretrained models to classify pet breeds from the Oxford-IIIT Pet Dataset. We'll compare different architectures and fine-tuning strategies to find the best approach.

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report

## 1.2 Set Up Environment and Reproducibility

Setting a random seed ensures that our results are reproducible across different runs.

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Set deterministic backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed()

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Data Loading and Preprocessing

We'll load the Oxford-IIIT Pet Dataset and apply appropriate transformations for transfer learning with pretrained models.

## 2.1 Define Data Transformations

For transfer learning, we need to use the normalization values that the pretrained models expect, which are typically the ImageNet statistics.

In [ ]:
# For transfer learning, we need to use the normalization values that the pretrained models expect
# These values are the mean and std from ImageNet dataset

# Transformations for training (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformations for validation/testing (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## 2.2 Load the Oxford-IIIT Pet Dataset

Let's load the dataset and create appropriate data loaders.

In [ ]:
# Load train+validation and test sets
trainval_dataset = datasets.OxfordIIITPet(root="./data", split="trainval", transform=train_transform, download=True)
test_dataset = datasets.OxfordIIITPet(root="./data", split="test", transform=val_transform, download=True)

# Split trainval into train and validation
train_size = int(0.8 * len(trainval_dataset))
val_size = len(trainval_dataset) - train_size
train_dataset, val_dataset = random_split(trainval_dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Get class information
class_to_idx = trainval_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
num_classes = len(class_to_idx)
print(f"Number of classes: {num_classes}")